In [1]:
import warnings

import pymc3 as pm
import numpy as np
import arviz as az
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score


RANDOM_SEED = 42
warnings.filterwarnings("ignore")

In [6]:
class BayesianExperiment:
    def __init__(
        self, 
        data, 
        target, 
        n_splits=5, 
        hdi_prob = 0.95,
        random_state=42, 
        n_iterations=20000, 
        tuning_iterations=12000, 
        probability_threshold=0.5,
        coverage_quantiles=[0.25, 0.5, 0.75]
    ):
        self.data = data
        self.target = target
        self.n_splits = n_splits
        self.hdi_prob = hdi_prob
        self.random_state = random_state
        self.n_iterations = n_iterations
        self.tuning_iterations = tuning_iterations
        self.probability_threshold = probability_threshold
        self.coverage_quantiles = coverage_quantiles
        
        self.features = data.drop(target, axis=1).columns.tolist()

    def predict_posterior(self, trace, X):
        X = X.copy()

        data = dict(intercept=trace['beta_i'])
        for f in self.features:
            data[f] = trace[f"beta_{f}"]

        coef_samples = pd.DataFrame(data)

        X.insert(0, "intercept", 1)
        linear_combinations = np.matmul(coef_samples, X.T)
        probabilities = 1 / (1 + np.exp(-linear_combinations))

        return probabilities

    def fit_model(self, X_train, y_train):
        lower = -10**6
        higher = 10**6

        with pm.Model() as model:
            beta_i = pm.Uniform('beta_i', lower=lower, upper=higher)

            betas = dict()
            for f in self.features:
                betas[f"beta_{f}"] = pm.Uniform(f"beta_{f}", lower=lower, upper=higher)

            linear_combination = beta_i
            for f in self.features:
                linear_combination += betas[f"beta_{f}"] * X_train[f]

            p = pm.Deterministic('p', pm.math.sigmoid(linear_combination))

            observed = pm.Bernoulli("target", p, observed=y_train)
            start = pm.find_MAP()
            step = pm.Metropolis()

            trace = pm.sample(self.n_iterations, step=step, start=start)
            burned_trace = trace[self.tuning_iterations:]

        return burned_trace

    def run(self):
        kf = KFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)

        X = self.data.drop(self.target, axis=1)
        y = self.data[self.target]

        metrics = {
            'overall_accuracy': [],
            'overall_f1_score': []
        }
        for q in self.coverage_quantiles:
            metrics[f"top_{int(q * 100)}_accuracy"] = []
            metrics[f"top_{int(q * 100)}_f1_score"] = []

        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y[train_index], y[test_index]

            trace = self.fit_model(X_train, y_train)
            posterior = self.predict_posterior(trace, X_test)

            bound = az.hdi(posterior.values, hdi_prob=self.hdi_prob)
            result = pd.DataFrame(dict(
                lb=bound[:, 0],
                ub=bound[:, 1],
                pred_proba=posterior.mean(),
                pred_class=(posterior.mean() > self.probability_threshold).astype(int),
                interval_width=bound[:, 1] - bound[:, 0],
                y_true=y_test.values
            ))

            accuracy = accuracy_score(result.y_true, result.pred_class)
            metrics["overall_accuracy"].append(accuracy)

            f1 = f1_score(result.y_true, result.pred_class)
            metrics["overall_f1_score"].append(f1)

            for q in self.coverage_quantiles:
                width_top_q = result.interval_width.quantile(q)
                reliable = result.loc[result.interval_width < width_top_q]

                accuracy = accuracy_score(reliable.y_true, reliable.pred_class)
                metrics[f"top_{int(q * 100)}_accuracy"].append(accuracy)

                f1 = f1_score(reliable.y_true, reliable.pred_class)
                metrics[f"top_{int(q * 100)}_f1_score"].append(f1)

        self.metrics_ = pd.DataFrame(metrics).describe().T[["mean", "std"]]
        self.trace_ = trace
        
        return self.metrics_

## Heart Attack

In [7]:
%%time

heart = pd.read_csv("../data/processed/heart.csv")

experiment = BayesianExperiment(
    data=heart, 
    target="HeartDisease",
    random_state=RANDOM_SEED
)
metrics = experiment.run()
metrics

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_ST_Slope]
>Metropolis: [beta_Oldpeak]
>Metropolis: [beta_ExerciseAngina]
>Metropolis: [beta_MaxHR]
>Metropolis: [beta_RestingECG]
>Metropolis: [beta_FastingBS]
>Metropolis: [beta_Cholesterol]
>Metropolis: [beta_RestingBP]
>Metropolis: [beta_ChestPainType]
>Metropolis: [beta_Sex]
>Metropolis: [beta_Age]
>Metropolis: [beta_i]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 110 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_ST_Slope]
>Metropolis: [beta_Oldpeak]
>Metropolis: [beta_ExerciseAngina]
>Metropolis: [beta_MaxHR]
>Metropolis: [beta_RestingECG]
>Metropolis: [beta_FastingBS]
>Metropolis: [beta_Cholesterol]
>Metropolis: [beta_RestingBP]
>Metropolis: [beta_ChestPainType]
>Metropolis: [beta_Sex]
>Metropolis: [beta_Age]
>Metropolis: [beta_i]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 109 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_ST_Slope]
>Metropolis: [beta_Oldpeak]
>Metropolis: [beta_ExerciseAngina]
>Metropolis: [beta_MaxHR]
>Metropolis: [beta_RestingECG]
>Metropolis: [beta_FastingBS]
>Metropolis: [beta_Cholesterol]
>Metropolis: [beta_RestingBP]
>Metropolis: [beta_ChestPainType]
>Metropolis: [beta_Sex]
>Metropolis: [beta_Age]
>Metropolis: [beta_i]


...

Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 100 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_ST_Slope]
>Metropolis: [beta_Oldpeak]
>Metropolis: [beta_ExerciseAngina]
>Metropolis: [beta_MaxHR]
>Metropolis: [beta_RestingECG]
>Metropolis: [beta_FastingBS]
>Metropolis: [beta_Cholesterol]
>Metropolis: [beta_RestingBP]
>Metropolis: [beta_ChestPainType]
>Metropolis: [beta_Sex]
>Metropolis: [beta_Age]
>Metropolis: [beta_i]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 98 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


CPU times: user 6min 32s, sys: 17.5 s, total: 6min 50s
Wall time: 12min 49s


,mean,std
overall_accuracy,0.844185,0.038276
overall_f1_score,0.857188,0.040303
top_25_accuracy,0.947826,0.024786
top_25_f1_score,0.960105,0.017565
top_50_accuracy,0.923435,0.037461
top_50_f1_score,0.935243,0.031626
top_75_accuracy,0.889506,0.052890
top_75_f1_score,0.900013,0.048942


## Australian Credit

In [8]:
%%time

australian = pd.read_csv("../data/processed/australian.csv")

experiment = BayesianExperiment(
    data=australian, 
    target="14",
    random_state=RANDOM_SEED
)
metrics = experiment.run()
metrics

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_13]
>Metropolis: [beta_12]
>Metropolis: [beta_11]
>Metropolis: [beta_10]
>Metropolis: [beta_9]
>Metropolis: [beta_8]
>Metropolis: [beta_7]
>Metropolis: [beta_6]
>Metropolis: [beta_5]
>Metropolis: [beta_4]
>Metropolis: [beta_3]
>Metropolis: [beta_2]
>Metropolis: [beta_1]
>Metropolis: [beta_0]
>Metropolis: [beta_i]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 122 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_13]
>Metropolis: [beta_12]
>Metropolis: [beta_11]
>Metropolis: [beta_10]
>Metropolis: [beta_9]
>Metropolis: [beta_8]
>Metropolis: [beta_7]
>Metropolis: [beta_6]
>Metropolis: [beta_5]
>Metropolis: [beta_4]
>Metropolis: [beta_3]
>Metropolis: [beta_2]
>Metropolis: [beta_1]
>Metropolis: [beta_0]
>Metropolis: [beta_i]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 118 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_13]
>Metropolis: [beta_12]
>Metropolis: [beta_11]
>Metropolis: [beta_10]
>Metropolis: [beta_9]
>Metropolis: [beta_8]
>Metropolis: [beta_7]
>Metropolis: [beta_6]
>Metropolis: [beta_5]
>Metropolis: [beta_4]
>Metropolis: [beta_3]
>Metropolis: [beta_2]
>Metropolis: [beta_1]
>Metropolis: [beta_0]
>Metropolis: [beta_i]


...

Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 117 seconds.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_13]
>Metropolis: [beta_12]
>Metropolis: [beta_11]
>Metropolis: [beta_10]
>Metropolis: [beta_9]
>Metropolis: [beta_8]
>Metropolis: [beta_7]
>Metropolis: [beta_6]
>Metropolis: [beta_5]
>Metropolis: [beta_4]
>Metropolis: [beta_3]
>Metropolis: [beta_2]
>Metropolis: [beta_1]
>Metropolis: [beta_0]
>Metropolis: [beta_i]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 114 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


CPU times: user 8min 2s, sys: 17.1 s, total: 8min 19s
Wall time: 15min 17s


,mean,std
overall_accuracy,0.863768,0.022569
overall_f1_score,0.849163,0.026687
top_25_accuracy,0.965714,0.046948
top_25_f1_score,0.946207,0.083531
top_50_accuracy,0.950725,0.012963
top_50_f1_score,0.929194,0.032434
top_75_accuracy,0.928155,0.018926
top_75_f1_score,0.912415,0.019025


## Diabetes

In [9]:
%%time

diabetes = pd.read_csv("../data/processed/diabetes.csv")

experiment = BayesianExperiment(
    data=diabetes, 
    target="Outcome",
    random_state=RANDOM_SEED
)
metrics = experiment.run()
metrics

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_Age]
>Metropolis: [beta_DiabetesPedigreeFunction]
>Metropolis: [beta_BMI]
>Metropolis: [beta_Insulin]
>Metropolis: [beta_SkinThickness]
>Metropolis: [beta_BloodPressure]
>Metropolis: [beta_Glucose]
>Metropolis: [beta_Pregnancies]
>Metropolis: [beta_i]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 65 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_Age]
>Metropolis: [beta_DiabetesPedigreeFunction]
>Metropolis: [beta_BMI]
>Metropolis: [beta_Insulin]
>Metropolis: [beta_SkinThickness]
>Metropolis: [beta_BloodPressure]
>Metropolis: [beta_Glucose]
>Metropolis: [beta_Pregnancies]
>Metropolis: [beta_i]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 71 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_Age]
>Metropolis: [beta_DiabetesPedigreeFunction]
>Metropolis: [beta_BMI]
>Metropolis: [beta_Insulin]
>Metropolis: [beta_SkinThickness]
>Metropolis: [beta_BloodPressure]
>Metropolis: [beta_Glucose]
>Metropolis: [beta_Pregnancies]
>Metropolis: [beta_i]


...

Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 66 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_Age]
>Metropolis: [beta_DiabetesPedigreeFunction]
>Metropolis: [beta_BMI]
>Metropolis: [beta_Insulin]
>Metropolis: [beta_SkinThickness]
>Metropolis: [beta_BloodPressure]
>Metropolis: [beta_Glucose]
>Metropolis: [beta_Pregnancies]
>Metropolis: [beta_i]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 66 seconds.
The rhat statistic is larger than 1.05 for some parameters. This indicates slight problems during sampling.
The estimated number of effective samples is smaller than 200 for some parameters.


CPU times: user 4min 34s, sys: 12.7 s, total: 4min 47s
Wall time: 23min 5s


,mean,std
overall_accuracy,0.768254,0.028765
overall_f1_score,0.633543,0.018254
top_25_accuracy,0.953576,0.033226
top_25_f1_score,0.832485,0.048686
top_50_accuracy,0.872180,0.034394
top_50_f1_score,0.801160,0.043976
top_75_accuracy,0.825507,0.020246
top_75_f1_score,0.743295,0.039607


## Qsar

In [10]:
%%time

qsar = pd.read_csv("../data/processed/qsar.csv")

experiment = BayesianExperiment(
    data=qsar, 
    target="Class",
    random_state=RANDOM_SEED
)
metrics = experiment.run()
metrics

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_V41]
>Metropolis: [beta_V40]
>Metropolis: [beta_V39]
>Metropolis: [beta_V38]
>Metropolis: [beta_V37]
>Metropolis: [beta_V36]
>Metropolis: [beta_V35]
>Metropolis: [beta_V34]
>Metropolis: [beta_V33]
>Metropolis: [beta_V32]
>Metropolis: [beta_V31]
>Metropolis: [beta_V30]
>Metropolis: [beta_V29]
>Metropolis: [beta_V28]
>Metropolis: [beta_V27]
>Metropolis: [beta_V26]
>Metropolis: [beta_V25]
>Metropolis: [beta_V24]
>Metropolis: [beta_V23]
>Metropolis: [beta_V22]
>Metropolis: [beta_V21]
>Metropolis: [beta_V20]
>Metropolis: [beta_V19]
>Metropolis: [beta_V18]
>Metropolis: [beta_V17]
>Metropolis: [beta_V16]
>Metropolis: [beta_V15]
>Metropolis: [beta_V14]
>Metropolis: [beta_V13]
>Metropolis: [beta_V12]
>Metropolis: [beta_V11]
>Metropolis: [beta_V10]
>Metropolis: [beta_V9]
>Metropolis: [beta_V8]
>Metropolis: [beta_V7]
>Metropolis: [beta_V6]
>Metropolis: [beta_V5]
>Metropolis: [beta_V4]
>Metropolis: [beta_V3]
>Metropolis: [b

Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 2106 seconds.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_V41]
>Metropolis: [beta_V40]
>Metropolis: [beta_V39]
>Metropolis: [beta_V38]
>Metropolis: [beta_V37]
>Metropolis: [beta_V36]
>Metropolis: [beta_V35]
>Metropolis: [beta_V34]
>Metropolis: [beta_V33]
>Metropolis: [beta_V32]
>Metropolis: [beta_V31]
>Metropolis: [beta_V30]
>Metropolis: [beta_V29]
>Metropolis: [beta_V28]
>Metropolis: [beta_V27]
>Metropolis: [beta_V26]
>Metropolis: [beta_V25]
>Metropolis: [beta_V24]
>Metropolis: [beta_V23]
>Metropolis: [beta_V22]
>Metropolis: [beta_V21]
>Metropolis: [beta_V20]
>Metropolis: [beta_V19]
>Metropolis: [beta_V18]
>Metropolis: [beta_V17]
>Metropolis: [beta_V16]
>Metropolis: [beta_V15]
>Metropolis: [beta_V14]
>Metropolis: [beta_V13]
>Metropolis: [beta_V12]
>Metropolis: [beta_V11]
>Metropolis: [beta_V10]
>Metropolis: [beta_V9]
>Metropolis: [beta_V8]
>Metropolis: [beta_V7]
>Metropolis: [beta_V6]
>Metropolis: [beta_V5]
>Metropolis: [beta_V4]
>Metropolis: [beta_V3]
>Metropolis: [b

Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 689 seconds.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_V41]
>Metropolis: [beta_V40]
>Metropolis: [beta_V39]
>Metropolis: [beta_V38]
>Metropolis: [beta_V37]
>Metropolis: [beta_V36]
>Metropolis: [beta_V35]
>Metropolis: [beta_V34]
>Metropolis: [beta_V33]
>Metropolis: [beta_V32]
>Metropolis: [beta_V31]
>Metropolis: [beta_V30]
>Metropolis: [beta_V29]
>Metropolis: [beta_V28]
>Metropolis: [beta_V27]
>Metropolis: [beta_V26]
>Metropolis: [beta_V25]
>Metropolis: [beta_V24]
>Metropolis: [beta_V23]
>Metropolis: [beta_V22]
>Metropolis: [beta_V21]
>Metropolis: [beta_V20]
>Metropolis: [beta_V19]
>Metropolis: [beta_V18]
>Metropolis: [beta_V17]
>Metropolis: [beta_V16]
>Metropolis: [beta_V15]
>Metropolis: [beta_V14]
>Metropolis: [beta_V13]
>Metropolis: [beta_V12]
>Metropolis: [beta_V11]
>Metropolis: [beta_V10]
>Metropolis: [beta_V9]
>Metropolis: [beta_V8]
>Metropolis: [beta_V7]
>Metropolis: [beta_V6]
>Metropolis: [beta_V5]
>Metropolis: [beta_V4]
>Metropolis: [beta_V3]
>Metropolis: [b

...

Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 1574 seconds.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_V41]
>Metropolis: [beta_V40]
>Metropolis: [beta_V39]
>Metropolis: [beta_V38]
>Metropolis: [beta_V37]
>Metropolis: [beta_V36]
>Metropolis: [beta_V35]
>Metropolis: [beta_V34]
>Metropolis: [beta_V33]
>Metropolis: [beta_V32]
>Metropolis: [beta_V31]
>Metropolis: [beta_V30]
>Metropolis: [beta_V29]
>Metropolis: [beta_V28]
>Metropolis: [beta_V27]
>Metropolis: [beta_V26]
>Metropolis: [beta_V25]
>Metropolis: [beta_V24]
>Metropolis: [beta_V23]
>Metropolis: [beta_V22]
>Metropolis: [beta_V21]
>Metropolis: [beta_V20]
>Metropolis: [beta_V19]
>Metropolis: [beta_V18]
>Metropolis: [beta_V17]
>Metropolis: [beta_V16]
>Metropolis: [beta_V15]
>Metropolis: [beta_V14]
>Metropolis: [beta_V13]
>Metropolis: [beta_V12]
>Metropolis: [beta_V11]
>Metropolis: [beta_V10]
>Metropolis: [beta_V9]
>Metropolis: [beta_V8]
>Metropolis: [beta_V7]
>Metropolis: [beta_V6]
>Metropolis: [beta_V5]
>Metropolis: [beta_V4]
>Metropolis: [beta_V3]
>Metropolis: [b

Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 694 seconds.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


CPU times: user 36min 28s, sys: 1min 1s, total: 37min 30s
Wall time: 2h 7min 19s


,mean,std
overall_accuracy,0.860664,0.013654
overall_f1_score,0.790248,0.021001
top_25_accuracy,0.969811,0.021513
top_25_f1_score,0.944788,0.050257
top_50_accuracy,0.975238,0.018565
top_50_f1_score,0.939621,0.053520
top_75_accuracy,0.925316,0.020703
top_75_f1_score,0.877517,0.049648


## Titanic

In [11]:
%%time

titanic = pd.read_csv("../data/processed/titanic.csv")

experiment = BayesianExperiment(
    data=titanic, 
    target="Survived",
    random_state=RANDOM_SEED
)
metrics = experiment.run()
metrics

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_Embarked_S]
>Metropolis: [beta_Embarked_Q]
>Metropolis: [beta_Embarked_C]
>Metropolis: [beta_Fare]
>Metropolis: [beta_Parch]
>Metropolis: [beta_SibSp]
>Metropolis: [beta_Age]
>Metropolis: [beta_Sex]
>Metropolis: [beta_Pclass]
>Metropolis: [beta_PassengerId]
>Metropolis: [beta_i]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 86 seconds.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_Embarked_S]
>Metropolis: [beta_Embarked_Q]
>Metropolis: [beta_Embarked_C]
>Metropolis: [beta_Fare]
>Metropolis: [beta_Parch]
>Metropolis: [beta_SibSp]
>Metropolis: [beta_Age]
>Metropolis: [beta_Sex]
>Metropolis: [beta_Pclass]
>Metropolis: [beta_PassengerId]
>Metropolis: [beta_i]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 87 seconds.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_Embarked_S]
>Metropolis: [beta_Embarked_Q]
>Metropolis: [beta_Embarked_C]
>Metropolis: [beta_Fare]
>Metropolis: [beta_Parch]
>Metropolis: [beta_SibSp]
>Metropolis: [beta_Age]
>Metropolis: [beta_Sex]
>Metropolis: [beta_Pclass]
>Metropolis: [beta_PassengerId]
>Metropolis: [beta_i]


...

Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 90 seconds.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [beta_Embarked_S]
>Metropolis: [beta_Embarked_Q]
>Metropolis: [beta_Embarked_C]
>Metropolis: [beta_Fare]
>Metropolis: [beta_Parch]
>Metropolis: [beta_SibSp]
>Metropolis: [beta_Age]
>Metropolis: [beta_Sex]
>Metropolis: [beta_Pclass]
>Metropolis: [beta_PassengerId]
>Metropolis: [beta_i]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 93 seconds.
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


CPU times: user 6min 2s, sys: 18.9 s, total: 6min 21s
Wall time: 12min 5s


,mean,std
overall_accuracy,0.797973,0.027867
overall_f1_score,0.727827,0.048508
top_25_accuracy,0.911111,0.062854
top_25_f1_score,0.897845,0.036375
top_50_accuracy,0.896629,0.032175
top_50_f1_score,0.845067,0.044961
top_75_accuracy,0.857356,0.030545
top_75_f1_score,0.790396,0.060896


In [12]:
print("Heart:", heart.shape)
print("Austalian:", australian.shape)
print("Titanic:", titanic.shape)
print("Diabetes:", diabetes.shape)
print("Qsar:", qsar.shape)

Heart: (918, 12)
Austalian: (690, 15)
Titanic: (891, 11)
Diabetes: (768, 9)
Qsar: (1055, 42)
